# Lab 2 - Python programming and data visualisation

## Problem 1

### Task

Fill in the Line class methods to accept coordinate as a pair of tuples and return the slope and distance of the line.

```Python
class Line(object):
    def __init__(self,coor1,coor2):
        """Initialize instance attributes with tuples (x1,y1)  and (x2,y2)
        """
        (...)

    def distance(self):
        """Calculate the length of the segment (line)
        """
        (...)

    def slope(self):
        """ Return the slope of a line going through the ends ( the 'a' in y=ax+b)
        """
        (...)
```

### Examples

```Python
coordinate1 = (3,2)
coordinate2 = (8,10)

li = Line(coordinate1,coordinate2)

li.distance()
9.433981132056603

li.slope()
1.6
```

In [21]:
import math

class Line(object):
    def __init__(self, coor1, coor2):
        self.p1 = coor1
        self.p2 = coor2
        
    def distance(self):
        difs = [self.p1[i] - self.p2[i] for i in range(2)]
        
        return math.hypot(*difs)
    
    def slope(self):
        difs = [self.p1[i] - self.p2[i] for i in range(2)]
        
#         return math.tan(math.atan2(*reversed(difs)))
        return difs[1] / difs[0]

    
coordinate1 = (3,2)
coordinate2 = (8,10)

li = Line(coordinate1,coordinate2)

li.distance()

li.slope()

1.6

## Problem 2

Fill in the class

```Python
class Cylinder(object):

    def __init__(self,height=1,radius=1):
        (...)

    def volume(self):
        (...)

    def surface_area(self):
        (...)
```

### Examples

```Python
c = Cylinder(2,3)

c.volume()
56.52

c.surface_area()
94.2
```

In [29]:
import math

class Cylinder(object):

    def __init__(self,height=1,radius=1):
        self.height = height
        self.radius = radius

    def volume(self):
        return math.pi * self.radius ** 2 * self.height

    def surface_area(self):
        base_area = math.pi * self.radius ** 2
        side_area = 2 * math.pi * self.radius * self.height
        return base_area * 2 + side_area
    

c = Cylinder(2,3)

c.volume()

c.surface_area()

94.24777960769379

## Problem 3

Write a class that takes a filename as an argument, open it and reads its content into a 2D matrix of values (an array of arrays: `[[1,2],[3,4]]`). The class should define a special function `info()` which prints out statistical information about column values in the file. Assume that the first row in the file specifies the column names

In [162]:
import statistics

class DataFile(object):
    def __init__(self, filename='undef'):
        with open(filename) as file:
            self.columns = list(map(str.strip, next(file).split(';')))
            
            self.data = []
            for line in file:
                separeated = map(str.strip, line.split(';'))
                self.data.append([int(f) if f.isdigit() else f for f in separeated])
  
    def info(self):
        result = []
        h = ['', 'Min', 'Max', 'Avg']
        header = f'{h[0]:8}{h[1]:^8}{h[2]:^8}{h[3]:^8}'
        result.append(header)
        
        for i, coulumn in enumerate(self.columns):
            name = coulumn + ':'
            min = self.min(i+1)
            max = self.max(i+1)
            avg = self.avg(i+1)
            min, max, avg = map(lambda x: '-' if x == None else round(x, 2), (min, max, avg))
            
            result.append(f'{name:8}{min:^8}{max:^8}{avg:^8}')
        
        result = '\n'.join(result)
        print(result)
    
    def _selectedColumn(self, colnum=0, colname='') -> int:
        if colnum == 0 and colname == '':
            raise ValueError('colnum or colname must be provided')
            
        column_number = self.columns.index(colname) + 1 if colname != '' else colnum
        if colnum == 0: colnum = column_number
        
        if colnum != column_number:
            raise ValueError(f'colnum selects {colnum} while colname selects {column_number} column')
            
        if (column_number < 1 or column_number > len(self.columns)):
            raise ValueError(f'Selected column number {column_number} is out of data range')
            
        return column_number - 1
        
    
    def avg(self, colnum=0, colname=''):
        """ The column name or colnum can be provided alternatively
        """
        index = self._selectedColumn(colnum, colname)
        values = (row[index] for row in self.data)
        values = list(filter(lambda x: type(x)==int, values))
        if len(values) <= 0: return None
        return statistics.mean(values)
        
    def min(self, colnum=0, colname=''):
        index = self._selectedColumn(colnum, colname)
        values = (row[index] for row in self.data)
        return min(filter(lambda x: type(x)==int, values), default=None)
      
    def max(self, colnum=0, colname=''):
        index = self._selectedColumn(colnum, colname)
        values = (row[index] for row in self.data)
        return max(filter(lambda x: type(x)==int, values), default=None)
    
data = DataFile('data.csv')

# print(data.avg(colname='Age'))
# print(data.avg(4))
data.info()

          Min     Max     Avg   
Name:      -       -       -    
Age:       4       8       6    
Weight:    18      32      25   
Height:    98     138    119.67 


## Problem 4

Write a context manager and example of its usage (just print some result to terminal) which will fetch the data about teachers from **ISOD API**, and as a variable it should return an **ordred by lastname** list of teachers.

In [177]:
import requests
import json
from contextlib import contextmanager

class ZETiSContext(object):
    def __enter__(self):
        s = requests.get("https://isod.ee.pw.edu.pl/isod-portal/wapi?q=list_teachers&sem2019Z&urlinfo=null&format=JSON&orgunit=ZETiIS")
        teachers = json.loads(s.text)['teachers']
        return sorted(teachers, key=lambda t: t['lastname'])

    def __exit__(self, type, value, traceback):
        return False

@contextmanager
def ZETiSContextDecorated():
    s = requests.get("https://isod.ee.pw.edu.pl/isod-portal/wapi?q=list_teachers&sem2019Z&urlinfo=null&format=JSON&orgunit=ZETiIS")
    teachers = json.loads(s.text)['teachers']
    yield sorted(teachers, key=lambda t: t['lastname'])

with ZETiSContext() as zetis:
    print(zetis[3])
    
with ZETiSContextDecorated() as zetis:
    print(zetis[5])


{'id': 2794, 'firstname': 'Bartosz', 'lastname': 'Chaber', 'email': 'bartosz.chaber@ee.pw.edu.pl', 'title': 'dr inż.', 'room': 'GE 223', 'www': 'www.goope.pw.edu.pl/bach', 'phone': '22 234 53 87', 'position': 'adiunkt', 'bio_pl': 'Komputerowe symulacje plazmy oraz pola elektromagnetycznego. Rozwijanie oprogramowania numerycznego oraz pomiary w zakresie kompatybilności elektromagnetycznej.', 'bio_en': 'Multilinguistic programming (Python, C, C++, Java, Ruby, Haskell), data visualization (OpenGL) and antennas analysis.', 'photourl': 'https://isod.ee.pw.edu.pl/isod-portal/photo/key/i2rkKMWoOWPeDOSjjlGjZQ.dat', 'orgunit': 'IETiSIP', 'orgunitfull': 'Instytut Elektrotechniki Teoretycznej i Systemów Informacyjno-Pomiarowych', 'facultyorgunit': 'EE', 'facultyorgunitfull': 'Wydział Elektryczny', 'universityorgunit': 'PW', 'universityorgunitfull': 'Politechnika Warszawska'}
{'id': 2474, 'firstname': 'Zygmunt', 'lastname': 'Filipowicz', 'email': 'zygmunt.filipowicz@ee.pw.edu.pl', 'title': 'doc. d